In [66]:
%load_ext chime
import chime
import rasterio as rio
import os
from glob import glob
from rasterio.windows import from_bounds
import numpy as np

The chime extension is already loaded. To reload it, use:
  %reload_ext chime


In [100]:
chime.theme("material")
loc_dem_2015 = "/mnt/PAG2015Elev/EPC_DEM_2015.vrt"
loc_dsms = glob("../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/*.tif")
loc_hags = "../EPCExtent_30cm/Elevation_80cmNPS/HAG_2015"
os.makedirs(loc_hags, exist_ok=True)

In [103]:
for loc_dsm in loc_dsms[:]:
    print(loc_dsm)
    with rio.open(loc_dsm) as src:
        bnds = src.bounds
        dsm = src.read(1)
        dsm_mask = dsm==src.nodata
        dsm[dsm_mask] = np.nan
        kwargs = src.profile
    try:
        with rio.open(loc_dem_2015) as src:
            dem = src.read(1, window=from_bounds(bnds.left, bnds.bottom, bnds.right, bnds.top, transform=src.transform), out_shape=(dsm.shape))
            dem_mask = dem==src.nodata
            dem[dem_mask] = np.nan
    except:
        print(f"Removing {loc_dsm}")
        loc_dsms.remove(loc_dsm)
        continue
    
    mask = dem_mask & dsm_mask
    hag = dsm - dem
    hag[mask] = np.nan
    if not np.isnan(np.nanmax(hag)):
        ohag = os.path.join(loc_hags, os.path.basename(loc_dsm).replace(".tif","_HAG.tif"))
        if not os.path.exists(ohag):
            print(f"Writing out HAG {ohag}")
            with rio.open(ohag, 'w', **kwargs) as dst:
                dst.write(hag, 1)
    else:
        print(f"Removing {loc_dsm}")
        loc_dsms.remove(loc_dsm)
        
%chime

../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E0860_N410_DSM80cm.tif
../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E0860_N570_DSM80cm.tif
../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E0880_N470_DSM80cm.tif
../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E0900_N370_DSM80cm.tif
../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E0920_N270_DSM80cm.tif


<ipython-input-103-8ca9a91f375e>:22: RuntimeWarning: All-NaN slice encountered
  if not np.isnan(np.nanmax(hag)):


Removing ../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E0920_N270_DSM80cm.tif
../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E0920_N570_DSM80cm.tif
Writing out HAG ../EPCExtent_30cm/Elevation_80cmNPS/HAG_2015/E0920_N570_DSM80cm_HAG.tif
../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E0940_N530_DSM80cm.tif
Writing out HAG ../EPCExtent_30cm/Elevation_80cmNPS/HAG_2015/E0940_N530_DSM80cm_HAG.tif
../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E0960_N490_DSM80cm.tif
Writing out HAG ../EPCExtent_30cm/Elevation_80cmNPS/HAG_2015/E0960_N490_DSM80cm_HAG.tif
../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E0980_N450_DSM80cm.tif
Writing out HAG ../EPCExtent_30cm/Elevation_80cmNPS/HAG_2015/E0980_N450_DSM80cm_HAG.tif
../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E1000_N410_DSM80cm.tif
Writing out HAG ../EPCExtent_30cm/Elevation_80cmNPS/HAG_2015/E1000_N410_DSM80cm_HAG.tif
../EPCExtent_30cm/Elevation_80cmNPS/DSM80cm/E1020_N370_DSM80cm.tif
Writing out HAG ../EPCExtent_30cm/Elevation_80cmNPS/HAG_2015/E1020_N370_DSM80cm_HA

In [104]:
len(loc_dsms)

193

In [105]:
loc_dsms_good = loc_dsms[:]